In [4]:
import heapq
from collections import namedtuple
from tqdm.auto import tqdm
import numpy as np
from random import choice


# Configuration
PUZZLE_DIM = 4
RANDOMIZE_STEPS = 10000
action = namedtuple('Action', ['pos1', 'pos2'])


# Define goal state and helpers
GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
GOAL_POSITIONS = {value: divmod(idx, PUZZLE_DIM) for idx, value in enumerate(GOAL_STATE.flatten())}

def available_actions(state: np.ndarray, blank_pos: tuple[int, int]) -> list['Action']:
    x, y = blank_pos
    actions = []
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


# Perform an action
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def state_to_bytes(state: np.ndarray) -> bytes:
    """Convert state to a bytes representation."""
    return state.flatten().tobytes()

def bytes_to_state(state_bytes: bytes) -> np.ndarray:
    """Convert bytes representation back to state."""
    return np.frombuffer(state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))



def weighted_manhattan_distance(state: np.ndarray, weights=(1.5, 1.0)) -> float:
    """Manhattan distance with weighted costs."""
    distance = 0
    for idx, value in enumerate(state.flatten()):
        if value != 0:
            current_pos = divmod(idx, PUZZLE_DIM)
            goal_pos = GOAL_POSITIONS[value]
            tile_weight = weights[1] if value < PUZZLE_DIM**2 // 2 else weights[0]
            distance += tile_weight * (abs(current_pos[0] - goal_pos[0]) + abs(current_pos[1] - goal_pos[1]))
    return distance

# A* Solver
def a_star_solver(start_state: np.ndarray):
    start_bytes = state_to_bytes(start_state)
    blank_idx = start_state.flatten().tolist().index(0)
    
    frontier = [(weighted_manhattan_distance(start_state), 0, start_bytes, blank_idx, None)]
    heapq.heapify(frontier)
    visited = {}
    parents = {}
    actions_evaluated = 0  # Counter for total actions evaluated

    while frontier:
        f_score, g_score, current_state_bytes, blank_idx, parent = heapq.heappop(frontier)
        
        if current_state_bytes in visited and visited[current_state_bytes] <= g_score:
            continue
        visited[current_state_bytes] = g_score
        parents[current_state_bytes] = parent
        
        current_state_np = bytes_to_state(current_state_bytes)
        blank_pos = divmod(blank_idx, PUZZLE_DIM)
        
        if current_state_np.tobytes() == GOAL_STATE.tobytes():
            return reconstruct_path(parents, current_state_bytes), actions_evaluated
        
        for action in available_actions(current_state_np, blank_pos):
            new_state_np = do_action(current_state_np, action)
            new_state_bytes = state_to_bytes(new_state_np)
            new_blank_idx = action.pos2[0] * PUZZLE_DIM + action.pos2[1]
            
            actions_evaluated += 1  # Increment counter for every action considered

            if new_state_bytes not in visited or visited[new_state_bytes] > g_score + 1:
                heapq.heappush(frontier, (
                    g_score + 1 + weighted_manhattan_distance(new_state_np),
                    g_score + 1,
                    new_state_bytes,
                    new_blank_idx,
                    current_state_bytes
                ))
    return None, actions_evaluated

def reconstruct_path(parents, state_bytes):
    """Reconstruct the path from start to goal."""
    path = []
    while state_bytes is not None:
        path.append(state_bytes)
        state_bytes = parents[state_bytes]
    return path[::-1]  # Reverse to get the path from start to goal


# Randomization
# Solve and report results
print("Start State:\n", state)
solution_path, total_actions_evaluated = a_star_solver(state)

if solution_path:
    print(f"Solution found in {len(solution_path) - 1} moves!")
    print("Final state after solving:")
    final_state = bytes_to_state(solution_path[-1])
    print(final_state)  # Print the final solved state
    # Calculate and print efficiency
    efficiency = (len(solution_path) - 1) / total_actions_evaluated if total_actions_evaluated > 0 else 0
    print(f"Total actions evaluated: {total_actions_evaluated}")
    print(f"Efficiency (solution moves / actions evaluated): {efficiency:.6f}")
else:
    print("No solution exists.")


Start State:
 [[ 9 15 12  3]
 [ 7 14 10  2]
 [ 0  5  8  1]
 [11  4  6 13]]
Solution found in 62 moves!
Final state after solving:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
Total actions evaluated: 66141419
Efficiency (solution moves / actions evaluated): 0.000001
